In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.4`
import $ivy.`sh.almond::almond-spark:0.6.0`
import $ivy.`io.projectglow::glow:0.2.0`
import io.projectglow.Glow
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import $ivy.$                                  

import $ivy.$                              

import $ivy.$                           

import io.projectglow.Glow

import org.apache.spark.sql._

import org.apache.spark.sql.functions._

In [2]:
val ss = {
  NotebookSparkSession
    .builder()
    .config("spark.sql.shuffle.partitions", "1")
    .config("spark.ui.enabled", "false")
    .config("spark.driver.host", "localhost")
    .master("local[*]")
    .getOrCreate()
}
import ss.implicits._
Glow.register(ss)

Loading spark-stubs


Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.11/0.5.0/spark-stubs_24_2.11-0.5.0.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.11/0.5.0/spark-stubs_24_2.11-0.5.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.11/0.5.0/spark-stubs_24_2.11-0.5.0.jar
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.11/0.5.0/spark-stubs_24_2.11-0.5.0.jar.sha1
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.11/0.5.0/spark-stubs_24_2.11-0.5.0-sources.jar.sha1
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.11/0.5.0/spark-stubs_24_2.11-0.5.0-sources.jar
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/eczech/.cache/coursier/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.16/slf4j-log4j12-1.7.16.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/eczech/.cache/coursier/v1/https/repo

Creating SparkSession


19:18:42.992 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


ss: SparkSession = org.apache.spark.sql.SparkSession@7de09306
import ss.implicits._


In [11]:
val df = ss.read.format("plink").load("/home/eczech/data/gwas/tutorial/1_QC_GWAS/HapMap_3_r3_1.bed")

df: DataFrame = [contigName: string, names: array<string> ... 6 more fields]

In [12]:
df.printSchema

root
 |-- contigName: string (nullable = true)
 |-- names: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- position: double (nullable = true)
 |-- start: long (nullable = true)
 |-- end: long (nullable = true)
 |-- referenceAllele: string (nullable = true)
 |-- alternateAlleles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genotypes: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- sampleId: string (nullable = true)
 |    |    |-- calls: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



In [13]:
df.count

count at cmd12.sc:1

8 / 8

count at cmd12.sc:1

1 / 1

res12: Long = 1457897L

In [7]:
df.groupBy(size($"genotypes.calls")).count.show

show at cmd6.sc:1

8 / 8

show at cmd6.sc:1

1 / 1

+---------------------+-------+
|size(genotypes.calls)|  count|
+---------------------+-------+
|                  165|1457897|
+---------------------+-------+



In [14]:
1457897 * 165

res13: Int = 240553005

In [16]:
df
    .withColumn("genotypes", explode($"genotypes"))
    .show(3, false)

show at cmd15.sc:3

1 / 1

+----------+-----------+--------+------+------+---------------+----------------+----------------------+
|contigName|names      |position|start |end   |referenceAllele|alternateAlleles|genotypes             |
+----------+-----------+--------+------+------+---------------+----------------+----------------------+
|1         |[rs2185539]|0.0     |556737|556738|C              |[T]             |[1328_NA06989, [0, 0]]|
|1         |[rs2185539]|0.0     |556737|556738|C              |[T]             |[1377_NA11891, [0, 0]]|
|1         |[rs2185539]|0.0     |556737|556738|C              |[T]             |[1349_NA11843, [0, 0]]|
+----------+-----------+--------+------+------+---------------+----------------+----------------------+
only showing top 3 rows



In [9]:
df
    .withColumn("genotypes", explode($"genotypes"))
    .groupBy(size($"genotypes.calls")).count.show

show at cmd8.sc:3

8 / 8

show at cmd8.sc:3

1 / 1

+---------------------+---------+
|size(genotypes.calls)|    count|
+---------------------+---------+
|                    2|240553005|
+---------------------+---------+



In [17]:
df
    .withColumn("genotypes", explode($"genotypes"))
    .groupBy(size($"names"), size($"alternateAlleles")).count.show

show at cmd16.sc:3

8 / 8

show at cmd16.sc:3

1 / 1

+-----------+----------------------+---------+
|size(names)|size(alternateAlleles)|    count|
+-----------+----------------------+---------+
|          1|                     1|240553005|
+-----------+----------------------+---------+



In [21]:
df
    .withColumn("genotypes", explode($"genotypes"))
    .withColumn("call", explode_outer($"genotypes.calls"))
    .withColumn("sampleId", $"genotypes.sampleId")
    .drop("genotypes")
    .show(3, false)

show at cmd20.sc:6

1 / 1

+----------+-----------+--------+------+------+---------------+----------------+----+------------+
|contigName|names      |position|start |end   |referenceAllele|alternateAlleles|call|sampleId    |
+----------+-----------+--------+------+------+---------------+----------------+----+------------+
|1         |[rs2185539]|0.0     |556737|556738|C              |[T]             |0   |1328_NA06989|
|1         |[rs2185539]|0.0     |556737|556738|C              |[T]             |0   |1328_NA06989|
|1         |[rs2185539]|0.0     |556737|556738|C              |[T]             |0   |1377_NA11891|
+----------+-----------+--------+------+------+---------------+----------------+----+------------+
only showing top 3 rows



In [23]:
df
    .withColumn("genotypes", explode($"genotypes"))
    .withColumn("call", explode($"genotypes.calls"))
    .withColumn("sampleId", $"genotypes.sampleId")
    .drop("genotypes")
    .groupBy("sampleId").agg(countDistinct($"names").as("n_snp"))
    .groupBy($"n_snp").count.show

show at cmd22.sc:7

1 + 7 / 8

: 

In [ ]:
   selectExpr("*", "expand_struct(call_summary_stats(genotypes))", "expand_struct(hardy_weinberg(genotypes))")